In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [4]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


9nd1XpfpRCjJpW9mdABeppNHcgRx7DXjMUBKGLxDR4Pg


In [5]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, qxVu9PGt8MHtvyDpDRTuPAvhXWKGcPnBNmHKDCg2B3Y, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [6]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


+----+----------------+----------------+---------+
|    | airport_code   | airport_name   | LABEL   |
|----+----------------+----------------+---------|
|  0 | MKE            | Milwaukee      | Airport |
|  1 | ORD            | Chicago O-Hare | Airport |
|  2 | SJC            | San Jose       | Airport |
|  3 | DEN            | Denver         | Airport |
+----+----------------+----------------+---------+


In [7]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

print(tabulate(df_flights, headers='keys', tablefmt='psql'))


+----+------------+----------+------------+------------+----------+
|    | START_ID   | END_ID   |   DISTANCE |   NUM_HOPS | TYPE     |
|----+------------+----------+------------+------------+----------|
|  0 | MKE        | ORD      |         66 |          1 | FLIES_TO |
|  1 | ORD        | MKE      |         66 |          1 | FLIES_TO |
|  2 | ORD        | DEN      |        886 |          1 | FLIES_TO |
|  3 | DEN        | ORD      |        886 |          1 | FLIES_TO |
|  4 | SJC        | DEN      |        948 |          1 | FLIES_TO |
|  5 | DEN        | SJC      |        948 |          1 | FLIES_TO |
|  6 | SJC        | ORD      |       1829 |          1 | FLIES_TO |
+----+------------+----------+------------+------------+----------+


In [ ]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiport set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


/opt/conda/lib/python3.8/site-packages/katana/remote/aio/import_data/dataframe_importer/__init__.py:242: UserWarning: unsupported dtype 'object' for 'airport_code' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")
/opt/conda/lib/python3.8/site-packages/katana/remote/aio/import_data/dataframe_importer/__init__.py:242: UserWarning: unsupported dtype 'object' for 'START_ID' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")
/opt/conda/lib/python3.8/site-packages/katana/remote/aio/import_data/dataframe_importer/__init__.py:242: UserWarning: unsupported dtype 'object' for 'END_ID' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")


 0 messages received [?msg/s ]

#  Add Restaurants ..

In [ ]:

#  Here we'll add stuff to the graph that some analytics will not like ..
#

l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query_unpaginated(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.airport_code     = 'SJC' AND m.restaurant_code  = 'PAP'
   CREATE (n) -[r: ATTRACTION { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query_unpaginated(l_query))


#  Diagnostics ..

In [ ]:

#  What's in the graph ..

l_result1 = my_graph.query_unpaginated("""
   MATCH (n) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))


In [ ]:

l_result2 = my_graph.query("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN r
   """)
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


In [ ]:

#  What's in the edge .. 

l_result = my_graph.query("""

   MATCH ( n )  - [r] ->  (m)
   RETURN ID(n) AS FROM, ID(m) AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS, TYPE(r) AS my_type, LABELS(n) AS FROM_LABELS, LABELS(m) AS TO_LABELS
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))
